## Render

I created this module because I wanted a simple way to describe how to render pydantic models. I almost always repeat the same logic across pydantic models. Maybe I even want the same on dataclasses.

To explore this space a bit, I've created `AutoViewModel`. This lets you update fields and have them update within the notebook. This is best viewed in a live iteration of a notebook.

In [5]:
from render import AutoViewModel
from time import sleep

class Countdown(AutoViewModel):
    time: float = 10

    def start(self):
        while self.time > 0:
            self.time -= 1
            sleep(1)

    def render(self):
        return f"{self.time} seconds left"

countdown = Countdown(time=5)

countdown.display()
countdown.start()


Countdown(display_id='4bd754da-8517-400e-95f6-8cb5b72164ea', time=0.0)

You can return a plain string or HTML in render.

In [6]:
from render import AutoViewModel
from time import sleep

class CountdownPretty(AutoViewModel):
    time: float = 10

    def start(self):
        while self.time > 0:
            self.time -= 1
            sleep(1)

    def render(self):
        return f"<b>{self.time}</b> seconds left"

countdown = CountdownPretty(time=3)

countdown.display()
countdown.start()


CountdownPretty(display_id='e3fe5b72-626b-4756-ba57-a53179ed6c5d', time=0.0)